In [5]:
import cv2
import numpy as np
import os
import zipfile

# 压缩文件路径和解压目标文件夹
zip_file_path = 'E:/study-master/NUSCampus/sem1/IND5003/code/IND5003Project/outfile.zip' 
output_folder = 'E:/study-master/NUSCampus/sem1/IND5003/code/IND5003Project/outfile'

# 创建输出文件夹（如果不存在）
os.makedirs(output_folder, exist_ok=True)

# 解压缩
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

# 获取解压文件夹中的PNG文件并排序
files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith('.png')])

# 计算光流并可视化的函数
def visualize_optical_flow(files, track_color=(0, 255, 0), forecast_color=(255, 0, 0)):
    track_points = []

    for i in range(1, len(files)):
        img_prev = cv2.imread(files[i-1], cv2.IMREAD_GRAYSCALE)
        img_next = cv2.imread(files[i], cv2.IMREAD_GRAYSCALE)

        # 检查图像是否加载成功
        if img_prev is None or img_next is None:
            print(f"Error loading images: {files[i-1]} or {files[i]}")
            continue

        # 检查图像尺寸并进行调整
        if img_prev.shape != img_next.shape:
            # 确定更大的尺寸
            target_size = (max(img_prev.shape[1], img_next.shape[1]), 
                           max(img_prev.shape[0], img_next.shape[0]))

            # 调整图像大小
            img_prev = cv2.resize(img_prev, target_size)
            img_next = cv2.resize(img_next, target_size)

        # 计算光流
        flow = cv2.calcOpticalFlowFarneback(img_prev, img_next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # 计算平均运动
        avg_movement = np.mean(flow, axis=(0, 1))
        track_points.append((avg_movement[0], avg_movement[1]))

        # 创建可视化图像，使用彩色图像
        flow_img = cv2.cvtColor(img_next, cv2.COLOR_GRAY2BGR)

        # 绘制历史轨迹
        for j in range(len(track_points)):
            x = int(track_points[j][0] * (j + 1))
            y = int(track_points[j][1] * (j + 1))
            cv2.circle(flow_img, (x, y), 3, track_color, -1)  # 绿色历史轨迹

        # 绘制预测路径
        if i == len(files) - 1:
            last_x, last_y = int(track_points[-1][0]), int(track_points[-1][1])
            forecast_x = last_x + int(avg_movement[0] * 5)
            forecast_y = last_y + int(avg_movement[1] * 5)
            cv2.line(flow_img, (last_x, last_y), (forecast_x, forecast_y), forecast_color, 2)  # 红色预测路径

        # 显示结果
        cv2.imshow('Storm Track and Forecast Path', flow_img)

        # 按键延迟，等待一定时间，按'q'键退出
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

# 运行可视化
visualize_optical_flow(files)